<a href="https://colab.research.google.com/github/apalibaeva/simulative-python-base/blob/main/%D0%93%D0%BB%D0%B0%D0%B2%D0%B0_10_%D0%A2%D1%80%D0%B5%D1%82%D0%B8%D0%B9_%D0%BA%D0%B5%D0%B9%D1%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 3 кейс

**В этом кейсе вы будете рассчитывать:**
* retention
* rolling retention
* lifetime
* churn rate
* mau
* wau
* dau

**Важно**

Перед началом решения задачи выполните следующую ячейку - в ней скачиваются нужные файлы

In [3]:
!wget https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv

!wget https://gist.github.com/Vs8th/aacb80595d1d6aaa2e31eb735f8bc644/raw/entries.csv

!wget https://gist.github.com/Vs8th/0e827e9a608117345dd6585ab81e8c86/raw/metrics.txt

--2024-10-21 20:01:12--  https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.github.com (gist.github.com)... 140.82.113.4
Connecting to gist.github.com (gist.github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv [following]
--2024-10-21 20:01:12--  https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14918 (15K) [text/plain]
Saving to: ‘registrations.csv’

registrations.csv   100%[===================>]  14.57K  --.-KB/s    in 0s      

2024-10-21 20:01:12 (33.5 M

Файлами для работы являются `registrations.csv` и `entries.csv`. В них хранятся данные о регистрациях пользователей и входа на платформу соответственно.

### **Посчитайте Retention 15 дня (в процентах) для пользователей, зарегистрированных в январе**

Cохраните результат в переменную `retention_15_day`

**Примечание:** результат округлите до 5 знаков после запятой

In [14]:
# Ваше решение
import csv
from datetime import datetime

reg_info = []
ent_info = []

with open('registrations.csv', 'r', newline='', encoding='utf-8') as reg_file:
  reg_reader = csv.DictReader(reg_file, delimiter=';')

  for row in reg_reader:
    reg_info.append(row)


with open('entries.csv', 'r', newline='', encoding='utf-8') as ent_file:
  ent_reader = csv.DictReader(ent_file, delimiter=';')

  for row in ent_reader:
    ent_info.append(row)


In [15]:
cnt_reg = 0
counted_users = []
for i in reg_info:
    if datetime.strptime(i['registration_date'], '%Y-%m-%d').month == 1: #and i['user_id'] not in counted_users:
        cnt_reg += 1
        counted_users.append(i['user_id'])

In [16]:
temp_dict = {item['user_id']: item for item in reg_info}

In [17]:
combined_info = []
for row in ent_info:
    if row['user_id'] in temp_dict:
        entry_date = datetime.strptime(row['entry_date'], '%Y-%m-%d')
        reg_date = datetime.strptime(temp_dict[row['user_id']]['registration_date'], '%Y-%m-%d')
        if reg_date.month == 1:
            addition = {**row, **temp_dict[row['user_id']], 'days_diff': (entry_date - reg_date).days}
            combined_info.append(addition)

In [18]:
cnt_15 = 0
counted_users = []
for item in combined_info:
    if item['days_diff'] == 15 and item['user_id'] not in counted_users:
        cnt_15 += 1
        counted_users.append(item['user_id'])

In [19]:
retention_15_day = round((cnt_15 / cnt_reg) * 100, 5)

In [20]:
cnt_reg, cnt_15, retention_15_day

(86, 47, 54.65116)

In [29]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
# Открываем файл с правильными ответами
with open('metrics.txt', 'r') as f:
    answers = f.read().split('\n')

correct_answer = float(answers[0])

try:
    assert retention_15_day == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


**Посчитайте Rolling-retention 30 дня (в процентах) для пользователей из той же когорты**

Сохраните результат в переменную `rolling_retention`

**Примечание:** результат округлите до 5 знаков после запятой

In [21]:
# Ваше решение
import csv
from datetime import datetime

reg_info = []
ent_info = []

with open('registrations.csv', 'r', newline='', encoding='utf-8') as reg_file:
  reg_reader = csv.DictReader(reg_file, delimiter=';')

  for row in reg_reader:
        user_id = row['user_id']
        reg_date = datetime.strptime(row['registration_date'], '%Y-%m-%d')
        if reg_date.month == 1:
            reg_info.append({'user_id': user_id, 'reg_date': reg_date})


with open('entries.csv', 'r', newline='', encoding='utf-8') as ent_file:
  ent_reader = csv.DictReader(ent_file, delimiter=';')

  for row in ent_reader:
        entry_date = datetime.strptime(row['entry_date'], '%Y-%m-%d')
        ent_info.append({'user_id': row['user_id'], 'entry_date': entry_date})



In [22]:
result = []
for i in reg_info:
    user_id = i['user_id']
    reg_date = i['reg_date']

    #есть дубли в датах по захода на одного пользователя, поэтому по каждому пользователю дату захода сохраняем в множество
    user_entries = set([entries['entry_date'] for entries in ent_info if entries['user_id'] == user_id])
    user_entries = list(user_entries)
    user_entries.sort()

    for entry in user_entries:
        days_diff = (entry - reg_date).days
        result.append({'user_id': user_id, 'reg_date': reg_date, 'days_diff': days_diff})

In [23]:
total_users = len(reg_info)
returned_users = len([item for item in result if item['days_diff'] >= 30])

In [24]:
rolling_retention = round(returned_users / cnt_reg * 100, 5)
rolling_retention

29.06977

In [30]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[1])

try:
    assert rolling_retention == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Lifetime по всем пользователям, посчитанный как интеграл от n-day retention**

Сохраните результат в переменную `lifetime`

**Примечание:** результат округлите до 5 знаков после запятой

In [25]:
reg_info = []

with open('registrations.csv', 'r', newline='', encoding='utf-8') as reg_file:
  reg_reader = csv.DictReader(reg_file, delimiter=';')

  for row in reg_reader:
        user_id = row['user_id']
        reg_date = datetime.strptime(row['registration_date'], '%Y-%m-%d')
        reg_info.append({'user_id': user_id, 'reg_date': reg_date})

total_users = len(reg_info)

# Ваше решение
result = {}

# сначала проходим по каждому пользователю и определяем его заход
for i in reg_info:
    user_id = i['user_id']
    reg_date = i['reg_date']

    # даты заходов пользователя
    user_entries = set([entries['entry_date'] for entries in ent_info if entries['user_id'] == user_id])
    user_entries = list(user_entries)
    user_entries.sort()

    # теперь считаем кол-во заходов на каждый день
    for entry in user_entries:
        days_diff = (entry - reg_date).days
        if days_diff not in result:
            result[days_diff] = 0
        result[days_diff] += 1

In [26]:
lifetime = 0
# считаем n-day retention на каждый день
for day, count in result.items():
    retention = count / total_users
    lifetime += retention

lifetime = round(lifetime, 5)
lifetime, total_users

(14.804, 1000)

In [31]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[2])

try:
    assert lifetime == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Churn rate 29 дня (в долях), посчитанный по всем пользователям**

Сохраните результат в переменную `churn_29`

In [ ]:
# Ваше решение

ottok = set()

# сначала проходим по каждому пользователю и определяем его заход
for i in reg_info:
    user_id = i['user_id']
    reg_date = i['reg_date']

    # даты заходов пользователя
    last_entry = max([entries['entry_date'] for entries in ent_info if entries['user_id'] == user_id])

    # теперь считаем кол-во заходов на каждый день
    if (last_entry - reg_date).days < 29:
        ottok.add(user_id)

churn_29 = len(ottok)/ total_users

In [ ]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[3])

try:
    assert churn_29 == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Mau, Wau, Dau за последний месяц/неделю/день записей**

Сохраните результат в переменные `dec_mau`, `dec_wau`, `dec_dau` соответственно

**Примечание:** последний месяц записей - декабрь. Поэтому `mau` рассчитываем для декабря (2021 года), для `wau` берем последнюю неделю - с 25 по 31 декабря, и для `dau` соответственно последний день - 31 декабря.

In [32]:
# Ваше решени(е
max_month = max(entry['entry_date'] for entry in ent_info).month

month_users = set(item['user_id'] for item in ent_info if item['entry_date'].month == max_month)
dec_mau = len(month_users)

In [33]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[4])

try:
    assert dec_mau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [34]:
# Ваше решение
from datetime import datetime, timedelta

max_day = max(entry['entry_date'] for entry in ent_info)
min_day = max_day - timedelta(days=6)

week_users = set(item['user_id'] for item in ent_info if min_day <= item['entry_date'] <= max_day )

dec_wau = len(week_users)
dec_wau

84

In [35]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[5])

try:
    assert dec_wau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [36]:
# Ваше решение
day_users = set(item['user_id'] for item in ent_info if item['entry_date'] == max_day )
dec_dau = len(day_users)

dec_dau

47

In [37]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[6])

try:
    assert dec_dau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Mau, Wau, Dau усредненные**

Сохраните результат в переменные `avg_mau`, `avg_wau`, `avg_dau` соответственно

**Примечание:** результаты округлите до 5 знаков после запятой

In [38]:
# Ваше решение
months = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
month_cnt = []
for month in months:
    count = len(set(item['user_id'] for item in ent_info if item['entry_date'].month == month))
    month_cnt.append(count)

avg_mau = round(sum(month_cnt)/len(month_cnt), 5)

avg_mau

102.58333

In [39]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[7])

try:
    assert avg_mau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [40]:
# Ваше решение
first_day = min(entry['entry_date'] for entry in ent_info)
last_day = max(entry['entry_date'] for entry in ent_info)

first_week_day = first_day.weekday()

if first_week_day == 1:
    current_day = first_day
else:
    days_to_monday = (0 - first_day.weekday()) % 7
    current_day = (first_day + timedelta(days=days_to_monday)) - timedelta(days=7)

weeks_report = {}

while current_day <= last_day:
    week_end = current_day + timedelta(days=6)
    count = len(set(item['user_id'] for item in ent_info if current_day <= item['entry_date'] <= week_end))
    weeks_report[f'{current_day} - {week_end}'] = count
    current_day = week_end + timedelta(days=1)

avg_wau = round(sum(weeks_report.values()) / len(weeks_report), 5)
avg_wau

89.86792

In [41]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[8])

try:
    assert avg_wau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [42]:
# Ваше решение
entries = set(item['entry_date'] for item in ent_info)

days_report = {}

for entry in entries:
    count = len(set(i['user_id'] for i in ent_info if i['entry_date'] == entry))
    days_report[f'{entry}'] = count

avg_dau = round(sum(days_report.values()) / len(days_report), 5)
avg_dau

40.5589

In [43]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[9])

try:
    assert avg_dau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!
